Halloween Challenge

This code was written in collaboration with Giacomo Cauda (https://github.com/jackcauda00/computational-intelligence)

In [2]:
from itertools import product
from random import random, randint, seed
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy


In [3]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [ ]:
NUM_SET = 5000
NUM_POINTS = NUM_SET
DENSITY = .7
count = 0

def fitness(state):
    #count the call to the fitness function
    global count
    count = count + 1

    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [[x[i, j] for j in range(NUM_POINTS)] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS - 1)
    new_state[index] = not new_state[index]
    return new_state

x = make_set_covering_problem(NUM_SET, NUM_POINTS, DENSITY)

current_state = [False for _ in range(NUM_SET)]

for step in range(1_000):
    new_state = tweak(current_state)
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state

print(count)
print(fitness(current_state))

tabu_list = []
count = 0
current_state = [False for _ in range(NUM_SET)]

for step in range(1_000):
    new_state = tweak(current_state)
    
    if not new_state in tabu_list: #tabu_list NON contiene new_state
        if fitness(new_state) >= fitness(current_state):
            tabu_list.append(current_state)
            current_state = new_state
        else:
            tabu_list.append(new_state)

print(count)
print(fitness(current_state))

We have done a comparison using the same data with our implementation and the hill climbing (HC):

| Parameters | Calls | Gained (comp. to HC) | Result (comp. to HC) |
|------------|-------|----------------------|----------------------|
|  100, .3   | 328   |          1672        |      Better          |
|  100, .7   | 386   |          1614        |       Same           |
|  1000, .3  | 1724  |          276         |       Same           |
|  1000, .7  | 1540  |          460         |      Better          |
|  5000, .3  | 1892  |          108         |      Better          |
|  5000, .7  | 1888  |          112         |       Same           |